<a href="https://colab.research.google.com/github/DavidMercadoFaustino/Kaggle/blob/master/Fake_News_Detection/Decision_tree_FAKE_NEWS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Importação de bibliotecas para utilização na Leitura e na manipulação de dados
import pandas as pd
import numpy as np
from collections import Counter

In [51]:
import re
import os
for dirname, _, filenames in os.walk('/content/Kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/Kaggle/test.csv
/content/Kaggle/submit.csv
/content/Kaggle/train.csv


In [52]:
# Carregamento pelo pandas por meio do read CSV
df_fake_news = pd.read_csv('/content/Kaggle/train.csv')

In [54]:
df_fake_news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
# Alterar os dados da coluna Label para melhor vizualização dos resultados
df_fake_news['label'].replace(1, 'News',regex=True, inplace=True) 
df_fake_news['label'].replace(0, 'Fake News',regex=True, inplace=True)
Counter(df_fake_news["label"])

Counter({'Fake News': 10387, 'News': 10413})

# Limpeza dos Dados

In [58]:
df_fake_news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [60]:
df_fake_news.dropna(axis=0, inplace=True)

In [61]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
list_stops_words = stopwords.words("english")
df_fake_news["title"] = df_fake_news['title'].str.lower()
df_fake_news["title"].replace('\n',' ', regex=True, inplace=True)

# Preparação dos dados

In [100]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_fake_news["title"], df_fake_news["label"], 
                                                    test_size=0.30, 
                                                    random_state=42)

In [87]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA

In [118]:
classify = Pipeline(
                [('vect', CountVectorizer(stop_words= list_stops_words,ngram_range=(1,2))), 
                 ('tfidf', TfidfTransformer()),
                 ('clf', DecisionTreeClassifier(criterion='entropy', splitter='random'))])

# Treinamento e Teste do modelo

In [119]:
# treinamento do nosso modelo
classify.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='entropy', max_depth=None,
                                        max_features=None, m

In [75]:
# Importação do metrics para medir o aprendizado da maquina
from sklearn import metrics

In [120]:
# Medição da precisão do modelo utilizando as amostras
classify.score(X_test, y_test)

0.9115931461903026

In [121]:
preds = classify.predict(X_test)
print(metrics.classification_report(y_test, preds))

              precision    recall  f1-score   support

   Fake News       0.95      0.90      0.92      3107
        News       0.87      0.93      0.90      2379

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486



# Agora usando o Texto

In [122]:
df_fake_news["text"] = df_fake_news['text'].str.lower()
df_fake_news["text"].replace('\n',' ', regex=True, inplace=True)

In [123]:

X_train, X_test, y_train, y_test = train_test_split(df_fake_news["text"], df_fake_news["label"], 
                                                    test_size=0.30, 
                                                    random_state=42)

In [124]:
classify = Pipeline(
                [('vect', CountVectorizer(stop_words= list_stops_words,ngram_range=(1,2))), 
                 ('tfidf', TfidfTransformer()),
                 ('clf', DecisionTreeClassifier(criterion='entropy', splitter='random'))
                 ])

In [125]:
classify.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='entropy', max_depth=None,
                                        max_features=None, m

In [127]:
classify.score(X_test, y_test)

0.8964637258476121

In [128]:
preds = classify.predict(X_test)
print(metrics.classification_report(y_test, preds))

              precision    recall  f1-score   support

   Fake News       0.90      0.91      0.91      3107
        News       0.89      0.87      0.88      2379

    accuracy                           0.90      5486
   macro avg       0.90      0.89      0.89      5486
weighted avg       0.90      0.90      0.90      5486

